# COMPARACIÓN AUTOMATIZADA DE PRECIOS DE PRODUCTOS DE PRIMERA NECESIDAD EN LOS 3 SUPERMERCADOS MÁS POPULARES: METRO, PLAZA VEA, TOTTUS.

### DESCRIPCIÓN DEL CASO

Se quiere automatizar la comparación de precios de productos básicos de la canasta familiar
(arroz, azúcar, aceite, leche, etc) de tres supermercados que ofrecen sus productos a través de
su propio portal web. Estos supermercados son: Metro, Plaza Vea y Tottus.
Por lo que se pide escribir un programa en Python utilizando Jupyter notebook que recopile los
precios de un producto de la canasta básica familiar de 3 Supermercados anteriormente
mencionados y reporte todos sus artículos relacionados con la palabra clave ingresada en la
búsqueda, para que posteriormente pueda comparar los precios en los distintos
Supermercados de los productos exactamente iguales. Además, que se envíe
automáticamente al correo del usuario-consultor un reporte de precio más alto, precio más
bajo, productos en común comparando los precios entre los supermercados coincidentes.

# Importar librerías


In [121]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pymongo import MongoClient
from tqdm import tqdm
from termcolor import colored
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
import time
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

# Documentación de la nueva actualización de Selenium


In [122]:
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"

# find_element(By.ID, "id")
# find_element(By.NAME, "name")
# find_element(By.XPATH, "xpath")
# find_element(By.LINK_TEXT, "link text")
# find_element(By.PARTIAL_LINK_TEXT, "partial link text")
# find_element(By.TAG_NAME, "tag name")
# find_element(By.CLASS_NAME, "class name")
# find_element(By.CSS_SELECTOR, "css selector")

# Programa para obtener los productos y precios de supermercados Metro (Usuario Jorge)

In [124]:
path = ('C:/Users/jorge/Documents/TRABAJO FINAL/msedgedriver.exe')  # Dirección del ejecutable chromedriver.exe
urls='https://www.metro.pe'          # URL de la pagina de Metro                   
options = webdriver.EdgeOptions()   
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
s = Service(path)                       
driver = webdriver.Edge(service=s, options = options)
driver.maximize_window()
driver.get(urls)

time.sleep(3) # lapso de 3 segundo para que la pagina cargue los contenidos

In [126]:
# Cancelar notificaciones, ventana emergente 
driver.find_element(By.XPATH, '/html/body/div[47]/div/div/div[2]/button[2]').click()
time.sleep(1)

In [127]:
# Hacer clic en "cerrar", pagina emergente que sale en inicio
driver.find_element(By.XPATH, '/html/body/div[46]/div/div[3]/button[1]').click()
time.sleep(1)

In [128]:
# Hacer clic en la barra de búsqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').click()
time.sleep(1)

In [129]:
busqueda=str(input())
# Ingresar el producto en la barra de busqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').send_keys(busqueda)
time.sleep(1)

azucar


In [130]:
# Dar clic en ver todos los resultados.
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[2]/div[2]/a/span').click()
time.sleep(1)

In [131]:
# Cantidad de productos
page=driver.find_element(By.XPATH, '/html/body/div[27]/div/div[2]/div[1]/div[1]/span[2]').text
int(page[:3])

164

In [132]:
# Automatización de scrolls (se correra las veces que sea necesario)
scrolls = 15 + 1           # Seran 15 scrolls automaticos
while True:               # Mientras esto sea verdad se realizará lo siguiente:
    scrolls -= 1           # Cuenta regresiva, se resta 1 cada ciclo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # Cada segundo se desplazará hasta la parte inferior de la página 
    time.sleep(1)      # Cada scroll automático tendra un timelap de 1 segundo
    if scrolls < 0:     # Cuando llegue a cero segundos:
        time.sleep(3)   # Le damos 3 segundos a que la pagina carge los contenidos hasta el ultimo scroll automatico
        driver.execute_script("window.scrollBy(0,-310)") # Un scroll up (rueda hacia arriba) para mostrar la ultima fila de productos
        elements = driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]') # Encontrar los productos de la pagina de acuerdo al XPATH
        break           # Se cierra el ciclo

        
print(colored('Productos extraidos:','red'))  
print(len(elements))        # Longitud de los elementos (productos) encontrados hasta coindicir con el total de productos

print(colored('\n Se llegó a extraer todos los productos?.','yellow'))  
print(len(elements) == int(page[:3])) # Comparación 

Productos extraidos:
164

 Se llegó a extraer todos los productos?.
True


In [133]:
# Vista a todos los elementos a scrapear, desde el primero hasta el ultimo en ser extradio
n = 1
while n < len(elements):  # Ciclo de acuerdo  a la cantidad de elementos extraidos
    listing = driver.find_element(By.XPATH, f'/html/body/div[27]/div/div[2]/div[4]/div[2]/div[2]/div[2]/div/ul/li[{n}]') 
    driver.execute_script("arguments[0].scrollIntoView(true);", listing)  # Se ira a la parte inferior hasta la ultima fila de productos
    n += 1
    

In [134]:
# Almacenar productos y precios 
productos =  []  # lista vacia
precios = []     # lista vacia
for element in tqdm(elements):  # tqdm:Contar iteraciones, indicar el tiempo que tarda por iteración y el tiempo total transcurrido.  
        name = element.find_element(By.XPATH,'.//a[@class="product-item__name"]').text  # Buscar producto en base al XPATH y mostrar el texto
        price = element.find_element(By.XPATH,'.//span[@class="product-prices__value product-prices__value--best-price"]').text # Buscar precio en base al XPATH y mostrar el texto
        productos.append(name)    # Agregar a la lista productos
        precios.append(price)  # Agregar a la lista precios


100%|██████████| 164/164 [00:15<00:00, 10.58it/s]


In [136]:
#Exportacion de productos y precios a un archivo excel 
dic= {'Producto': productos , "Precio": precios}  # Crear diccionario con los values productos y precios (listas)
df = pd.DataFrame(dic , columns= ["Producto", "Precio"] ) # Convertir a dataframe con columnas de etiquetas Producto y Precio
df.to_excel('metro.xlsx', index=False,encoding='cp1252')  # Exportar a excel, sin indices, y el encoding cp1252

# Lectura y muestra de resultados
print(colored('Imprimiendo resultados.','blue'))  
pd.read_excel('metro.xlsx')

Imprimiendo resultados.


,Producto,Precio
0,Azúcar Rubia Paramonga 1kg,S/4.95
1,Azúcar Morena Light Incauca 850g,S/15.85
2,Azúcar Blanca Light Incauca 850g,S/15.85
3,Azúcar Rubia Costeño 1 kg,S/5.90
4,Azúcar Impalpable Universal 200 g,S/3.10
...,...,...
159,Endulzante en Polvo Stevia Cuisine & Co 50g,S/18.90
160,Endulzante en Polvo Stevia Cuisine & Co 50un,S/12.20
161,Infusión Orgánica Diet Sunka Caja 21 unid,S/6.50
162,Endulzante En Polvo Vida Stevia 180 g,S/47.69


In [137]:
driver.close()  # Cerrar chromedriver

# Programa para obtener los productos y precios de supermercados Tottus  (Usuario Christian)

In [141]:
path = ("C:/Users/jorge/Documents/TRABAJO FINAL/msedgedriver.exe")
urls='https://www.tottus.com.pe/'
options = webdriver.EdgeOptions()
options.add_argument('--incognito')
s = Service(path)
driver = webdriver.Edge(service=s, options = options)
driver.maximize_window()
driver.get(urls)
time.sleep(3)

In [142]:
#Al iniciar y entrar a la pagina de tottus nos salta un anuncio donde tottus nos quiere informar sobre sus mejores ofertas,
#con este codigo automatizamos dando click en cancelar. 
WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      'button.align-right secondary slidedown-button'.replace(' ', '.'))))\
    .click()

In [143]:
# Configurando click para búsqueda de productos
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').click()
time.sleep(1)

In [144]:
# Ingresando producto 
busqueda=str(input())
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').send_keys(busqueda)
time.sleep(1)

azucar


In [146]:
# Buscar producto mediante el click al icono de busqueda
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div[1]/button').click()
time.sleep(1)

In [147]:
#Funcion para obtener el número de paginas
WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.XPATH,
                                      '/html/body/div[1]/section/div[1]/section/div[2]/div[2]/ul[1]')))


pag = driver.find_element("xpath",'/html/body/div[1]/section/div[1]/section/div[1]/div[2]/ul/li[5]')
pag=pag.text
def n_pag(pag):
    if pag=="Siguiente":
        pag = driver.find_element("xpath",'/html/body/div[1]/section/div[1]/section/div[1]/div[2]/ul/li[4]')
        pag=pag.text
        print(pag)
    else:
        if int(pag)>3:
            pag = driver.find_element("xpath",'/html/body/div[1]/section/div[1]/section/div[1]/div[2]/ul/li[5]')
            pag=pag.text
            print(pag)
        else:
            if pag==3:
                pag = driver.find_element("xpath",'/html/body/div[1]/section/div[1]/section/div[1]/div[2]/ul/li[4]')
                pag=pag.text
                print(pag.text)
            else:
                if pag==2:
                    pag= driver.find_element("xpath",'/html/body/div[1]/section/div[1]/section/div[1]/div[2]/ul/li[3]')
                    pag=pag.text
                    print(pag.text)
                else:
                    pag= driver.find_element("xpath",'/html/body/div[1]/section/div[1]/section/div[1]/div[2]/ul/li[2]')
                    pag=pag.text
                    print(pag.text)
# Aca se obtiene el numero de paginas de la busqueda realizada
n_pag(pag)

4


In [148]:
busqueda = input()
list= []
productos = []
precios =[]
n=1
while n <= int(pag):
    driver.get(f'https://www.tottus.com.pe/buscar?q={busqueda}&page={n}')
    time.sleep(5)
    lista = driver.find_elements(By.XPATH,'//li[@class="product"]')
    for element in tqdm(lista):
        name = element.find_element(By.XPATH,'.//h2[@class="jsx-369422875 name title mini"]').text
        price = element.find_element(By.XPATH,'.//span[@class="jsx-4019956066 "]').text
        productos.append(name)
        precios.append(price)

    lista=list.append(lista)
    n += 1

azucar


100%|██████████| 40/40 [00:02<00:00, 15.78it/s]


In [149]:
len(list)

4

In [150]:
dic= {'Producto': productos,"Precio": precios }
df = pd.DataFrame(dic , columns= ["Producto","Precio"] )
df.to_excel('tottus.xlsx', index=False,encoding='cp1252')

# Lectura
pd.read_excel('tottus.xlsx')

,Producto,Precio
0,AZÚCAR RUBIA TOTTUS 5 KG,s/ 19.9 UN
1,AZÚCAR RUBIA DULFINA 5 KG,s/ 22.9 UN
2,AZÚCAR RUBIA TOTTUS 1 KG,s/ 4.7 UN
3,AZÚCAR RUBIA TOTTUS 2 KG,s/ 8.5 UN
4,AZÚCAR RUBIA DULFINA 1 KG,s/ 5.2 UN
...,...,...
179,TRIPACK GASEOSA COCA COLA + INCA KOLA + FANTA ...,s/ 23.9 UN\ns/ 24.9 UN\ns/ 27 UN
180,TWO PACK COCA COLA + INCA KOLA 3 LT,s/ 19.5 UN\ns/ 19.9 UN\ns/ 20.2 UN
181,MASCARILLA FACIAL PULIDORA DE CARBÓN Y AZÚCAR ...,s/ 8.4 UN
182,ESTOFADO DE POLLO Y PURÉ + COCA COLA ZERO 300 ML,s/ 13.5 UN


In [151]:
driver.close()  # Cerrar chromedriver

# Programa para obtener los productos y precios de supermercados Tottus  (Usuario Brian)

In [201]:
import csv #importando codigos
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [202]:
driver = webdriver.Chrome()                 #probando driver
url='https://www.plazavea.com.pe'            #probando url

def busqueda(search):                       #definiendo funcion search
    "generar url"
    template = 'https://www.plazavea.com.pe/{}'
    search = search.replace('','')
    return template.format(search)  

In [203]:
url = busqueda('packs')
print(url)

https://www.plazavea.com.pe/packs


In [204]:
driver.get(url) #almacenandor driver

In [205]:
soup = BeautifulSoup(driver.page_source,'html.parser') #transformando a tipo BeatifulSoup

In [206]:
def extraerprecio(item): #definiendo funcion para extraer el precio
    #precio
    precio = item.find('div', 'Showcase__salePrice') #scrapeando html
    precio1 = precio.text.split() #limpiando el precio
    nombre=item.find('a','Showcase__name').text #limpiando el nombre del item
    #descripcion
    descripcion= (precio1, nombre)
    return descripcion             

In [207]:
records = []
resultados= soup.find_all('div','Showcase__content')

for item in resultados:
    record = extraerprecio(item)

    if record:
        records.append(record)

In [208]:
def busqueda(busqueda):
    "generar url"
    template = 'https://www.plazavea.com.pe/{}'
    busqueda = busqueda.replace('','')
    url=template.format(search)
    url += '?page={}'
    return url

In [209]:
#escribiendo el codigo para la busqueda
records = []
def busqueda(search):     #Creando función para buscar producto
    "generar url"         
    template = 'https://www.plazavea.com.pe/{}'
    search = search.replace('','')
    return template.format(search)  
def main(term):
    driver = webdriver.Chrome()# abriendo driver
    record = [] #almacenando resultados
    url= busqueda(term) 

    for page in range(1,9): #definiendo funcion para pasar pagina
        time.sleep(2)
        driver.get(url.format(page))
        soup= BeautifulSoup(driver.page_source,'html.parser') #escrapeando html
        resultados= soup.find_all('div','Showcase__content') #escrapeando html

        for item in resultados:
            record = extraerprecio(item) # recall extrayendo precios

            if record:
                records.append(record)
driver.close()

In [211]:
main('azucar') #utilizando funcion final

In [216]:
with open("plazavea.csv", 'w', newline='', encoding='utf-8') as f: # almacenando resultados
    writer = csv.writer(f)
    writer.writerow(['precio','descripción'])
    writer.writerows(records)

In [223]:
df = pd.read_csv('plazavea.csv')
print(df)

              precio                                        descripción
0    ['S/', '16.70']                      Azúcar Rubia BELL'S Bolsa 4Kg
1       ['S/', '23']            Azúcar Rubia BELL'S Selección Bolsa 5Kg
2    ['S/', '12.59']  Bebida de Almendra NATURE'S HEART Sin Azúcar C...
3    ['S/', '17.30']     Yogurt Griego TIGO Natural Sin Azúcar Pote 1Kg
4     ['S/', '7.90']        Azucarero VIVA HOME Porcelana Blanca Puntos
..               ...                                                ...
187   ['S/', '5.90']                     Azúcar Rubia COSTEÑO Bolsa 1Kg
188  ['S/', '11.80']  Gaseosa INCA KOLA Sin Azúcar Botella 1.5L Paqu...
189     ['S/', '13']  Gaseosa COCA COLA Sin Azúcar 6 Pack Botella 50...
190   ['S/', '4.80']                     Azúcar Rubia DULFINA Bolsa 1Kg
191      ['S/', '8']          Gaseosa COCA COLA sin Azúcar Botella 2.5L

[192 rows x 2 columns]


In [224]:
plazaveafinal = df.iloc[70:]
plazaveafinal.to_excel('salida.xlsx')

In [225]:
plazavea = pd.read_excel('salida.xlsx')
plazavea

,Unnamed: 0,precio,descripción
0,70,"['S/', '4.80']",Azúcar Rubia DULFINA Bolsa 1Kg
1,71,"['S/', '8.00']",Gaseosa COCA COLA sin Azúcar Botella 2.5L
2,72,"['S/', '16.70']",Azúcar Rubia BELL'S Bolsa 4Kg
3,73,"['S/', '23.00']",Azúcar Rubia BELL'S Selección Bolsa 5Kg
4,74,"['S/', '12.59']",Bebida de Almendra NATURE'S HEART Sin Azúcar C...
...,...,...,...
117,187,"['S/', '5.90']",Azúcar Rubia COSTEÑO Bolsa 1Kg
118,188,"['S/', '11.80']",Gaseosa INCA KOLA Sin Azúcar Botella 1.5L Paqu...
119,189,"['S/', '13']",Gaseosa COCA COLA Sin Azúcar 6 Pack Botella 50...
120,190,"['S/', '4.80']",Azúcar Rubia DULFINA Bolsa 1Kg


# Comparar productos de acuerdo a un búsqueda en común (FASE de prueba)

In [152]:
tottus = pd.read_excel('tottus.xlsx')
tottus[tottus['Producto'].str.count(r'(AZÚCAR.*RUBIA.*5.*KG.*)')==1]

,Producto,Precio
0,AZÚCAR RUBIA TOTTUS 5 KG,s/ 19.9 UN
1,AZÚCAR RUBIA DULFINA 5 KG,s/ 22.9 UN


In [153]:
metro=pd.read_excel('metro.xlsx')   
metro
metro[metro['Producto'].str.count(r'(Azúcar.*Rubia.*5.*kg.*)')==1]

,Producto,Precio
6,Azúcar Rubia Paramonga Bolsa 5 kg,S/18.90
13,Azúcar Rubia Máxima Bolsa 5 kg,S/19.95
14,Azúcar Rubia Metro Bolsa 5 kg,S/20.45
17,Azúcar Rubia Dulfina Bolsa 5 kg,S/21.95
21,Azúcar Rubia Paramonga Bolsa 5 kg,S/18.90


In [230]:
plazavea[plazavea['descripción'].str.count(r'(Azúcar.*Rubia.*5.*)')==1]

,Unnamed: 0,precio,descripción
3,73,"['S/', '23.00']",Azúcar Rubia BELL'S Selección Bolsa 5Kg
12,82,"['S/', '21.90']",Azúcar Rubia DULFINA Bolsa 5Kg
19,89,"['S/', '3.10']",Azúcar Rubia BELL'S Bolsa 650g
27,97,"['S/', '23.00']",Azúcar Rubia BELL'S Selección Bolsa 5Kg
36,106,"['S/', '21.90']",Azúcar Rubia DULFINA Bolsa 5Kg
43,113,"['S/', '3.10']",Azúcar Rubia BELL'S Bolsa 650g
51,121,"['S/', '23.00']",Azúcar Rubia BELL'S Selección Bolsa 5Kg
60,130,"['S/', '21.90']",Azúcar Rubia DULFINA Bolsa 5Kg
67,137,"['S/', '3.10']",Azúcar Rubia BELL'S Bolsa 650g
75,145,"['S/', '23']",Azúcar Rubia BELL'S Selección Bolsa 5Kg


# Envío de correo automático (FASE de prueba)

In [47]:
import smtplib  # Importar la librería

de_email ="okoyekennethoptimizer@gmail.com"   # Correo de envio (emisor)
contraseña = "nmmrjdphptwfgjgb"               # contraseña del emisor
para_email = "20200338@LAMOLINA.EDU.PE"       # Email de destino
asunto = "TRABAJO FINAL LP2"                  # Asunto de llegada
text ="""                                    
Hola estimada profesora, le enviamos las comparaciones de los productos.
Codigo de integrantes: 
"""                                           # Contenido del mensaje

mensaje = f'Subject: {asunto}\n\n{text}\n\n'  # Formato de llegada del mensaje
connection = smtplib.SMTP("smtp.gmail.com", port= 587) # Conexión al dominio gmail.com con puerto = 587
connection.starttls()             # Empezar  conexión            
connection.login(user=de_email, password=contraseña) # Logeando con correo y contraseña
connection.sendmail(from_addr=de_email, to_addrs=para_email, msg= mensaje) # Enviar email a destinatario
connection.close()                # Cerrar conexión    